In [8]:
class Vector(object):
    def __init__(self, coordinates):
        try:
            if not coordinates:
                raise ValueError
            self.coordinates = tuple(coordinates)
            self.dimensions = len(coordinates)

        except ValueError:
            raise ValueError("The coordinates must be non empty")

        except TypeError:
            raise TypeError("The coordinates must be an iterable")

    def __str__(self):
        return 'Vector {}'.format(self.coordinates)

    def __eq__(self, other):
        return self.coordinates == other.coordinates

In [9]:
Vector([1,2,3,4]) == Vector([1,2,3,4])

True

In [10]:
Vector([1,2,3,4]) == Vector([-1,2,3,4])

False

In [11]:
print(Vector([3,4,5]))

Vector (3, 4, 5)
